# Outbreak Detection in Networks

### Plan for Today
    - 1) new problem Outbreak detection
    - 2) Develop an approximation algorithm (It is a submodular opt.problem)
    - 3) Speed-up greedy hill-climbing (valid for optimizing general submodular functions)
    - 4) Prove a new "data dependent" bound on the solution quality (valid for optimizing general submodular functions)
    

## Detecting Contamination Outbreaks
Outbreak : 발생, 발발, 돌발 (출처 : Wiki)
 - 어떤 사건의 발생을 예측,감시하는 시스템
 - ex)
 어느 마을에는 수도관이 밑의 그림처럼 복잡하게 연결되어있다. 이 때 특정 노드에서 수질감염이 일어났다고 하자. 수도관이 연결되어있기 때문에 오염된 물은 퍼져나갈 것이고, 그림에서는 이를 빨간 색 영역으로 표시되어있다.
모든 집집마다 센서를 달아서 당장 어느 집이 문제인지 예측하면 좋으련만, 실제로는 몇 천, 몇 만이 될지 모르는 모든 수도관 하나하나마다 그 일을 할 수 있을까? 그래서 어디에 센서를 달아야 가장 효율적으로 Outbreak Detection을 할 수 있는지 알아보자는 것이 주제이다.
<img src ='attachment:image.png' width = 40%>

### General Problem
 - Dyanmic process spreading하는 graph가 주어졌을 때, process를 효율적으로 detect하는 역할을 해낼 Node의 집합을 구하자
 - Many other applications
     - Epidemics
     - Influence propagation
     - Network security 
     
### Water Network : Utility
 - 마을의 수도관 그래프가 주어졌다고 하자. 수도관은 현재 Dynamic process spreading을 하기 때문에 어느 지점에서 오염이 일어났느냐에 따라 전파되는 구간이 저마다 다르다. 
 <img src = 'attachment:image-2.png' width = 40%>
 - 두번째 그림을 보면 빨간색에 지역에 오염이 많이 발생 할 것으로 예상이 된다. 이를 보아 이곳에 센서를 설치하면 효율적으로 Outbreak Detection이 가능 할것이라고 예상됨.
 

### problem setting : Contamination(오염)
 - Given:
     - Graph G(V,E)
     - Data about how outbreaks spread over the G:
         - for each outbreak i we know the time T(u,i) when outbreak i contaminates node u
 - Goal : Select a subset of nodes S that maximizes the expected reward
     - <img src = "attachment:image-3.png" width = 40%>
     - B = budget  
     
 - Two parts to the problem
     - Reward (one of the following three)
         - 1) Minimize time to detecting
         - 2) Maximize number of detected propagations
         - 3) Minimize number of ingfecter people
     - Cost(context dependent): 연결이 너무 많으면 탐색에 시간이 오래 걸림
         - Reading big blogs is more time consuming
         - Placing a sensor in a remote location is expensive
       <img src = "attachment:image-4.png" width = 40%>

### Objectives for Outbreak Detection
 - Penalty $\pi_i(t)$ for detecting outbreak i at time t
     - 1) Time to detection (DT)
         - How long does it take to detect a contamination?
         - Penalty for detectng at time t: $\pi_i(t) = t$ 
     - 2) Detection likelihood (DL)
         - How many contaminations do we detect?
         - Penalty for detecting at time t:$\pi_i(t) = 0, \pi_i(inf) = 1$ 
     - 3) Population affected (PA)
         - How many people drank contaminated water?
         -  Penalty for detecting at time t:$\pi_i(t) =$ (# of infected nodes in outbreak i by time t)
         
 - Observation : In all detecting sooner does not hurt! (어떤 목적함수를 쓰든 시간이 얼마나 적게 걸리든 손해가 없음)
 
### Structure of the Problem
 - 앞서 배운 목적함수를 그대로 사용하는 것이 아니라 penalty reduction 함수 f를 정의하여 사용.
 - we define $f_i(S)$ as penalty reduction: $f_i(S) = \pi_i(inf) - \pi_i(T(S,i))$ (Submodular)
 - <img src = "attachment:image-5.png" width = 40%>
 
### Objective functions are Submodular
 - <img src = "attachment:image-6.png" width = 40%>
 - <img src = "attachment:image-7.png" width = 40%>
 - 1) T(B i) <= T(A, i) < T(x, i) : x가 가장 늦게 발견. 센서 x가 무쓸모!
 - 2) T(B i) <= T(x, i) <= T(A, i) : x가 B와 A사이. A케이스보다 나은 상황을 만들어주지만, B의 경우에서는 아니라는 것.
 - 3) T(x, i) < T(B, i) <= T(A, i) : x가 제일 빨리 발견함. 
 - A는 B의 subset이므로 항상 B가 A보다 앞에 있음!
 - Submodular의 combination인 f(S)도 똑같은 Submodular이다.
 - 이러한 3가지 케이스를 살펴보았을 때 우리의 목적함수는 Submodular이다!
 - 우리의 Submodular를 최적화하는 전통적 방법은 Hill-climbing이다 저번 강의에서 소개되었는데, 이 방법은 2가지 문제점이있다.
     - unit cost case(모든 센서의 같은 cost적용)에서만 작동한다. 저번 강의에서는 k명의 influencer를 골랐듯 sensor s의 cost c(s) 같은 사례에 적용하는 거다.
     - Hill-climbing is slow. 매 반복마다 모든 노드 marginal gain을 계산해야한다.$O(|V|K)$ for placing K sensors